<a href="https://colab.research.google.com/github/narevignesh/Multi-class-animal-recognition/blob/main/Multi_class_animal_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %%
# Purpose: Install necessary libraries for the project.
# Result: Installs the specified libraries and their dependencies.
!pip install kaggle
!pip install ipython
!pip install matplotlib
!pip install numpy
!pip install tensorflow
!pip install scikit-learn
!pip install Pillow

In [ ]:
# %%
# Purpose: Import necessary libraries for interacting with the notebook and displaying content.
# Result: Makes the functionalities of IPython and display available for use in the notebook.
from IPython import get_ipython
from IPython.display import display

In [ ]:
# %%
# Purpose: Download the animal image dataset from Kaggle.
# Result: Downloads the dataset and prints the path to the downloaded files.
import kagglehub

path = kagglehub.dataset_download("iamsouravbanerjee/animal-image-dataset-90-different-animals")

print("Path to dataset files:", path)

In [ ]:
# %%
# Purpose: Import necessary libraries for data processing, visualization, and model building.
# Result: Makes the functionalities of the imported libraries available for use in the notebook.
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# %%
# Purpose: Check and print the number of available GPUs.
# Result: Prints the number of available GPUs and the name of the GPU if available.
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

if tf.config.list_physical_devices('GPU'):
    print("GPU Name:", tf.config.list_physical_devices('GPU')[0].name)

In [ ]:
# %%
# Purpose: Define the dataset path and retrieve class information.
# Result: Sets the dataset path and prints the class names and the number of classes.
dataset_path = "/kaggle/input/animal-image-dataset-90-different-animals/animals/animals"
classes = os.listdir(dataset_path)
num_classes = len(classes)
print("Classes:", classes)
print("Number of classes:", num_classes)

In [ ]:
# %%
# Purpose: Display sample images from the dataset.
# Result: Generates a plot containing sample images from the first 10 classes, with their corresponding class names as titles.
plt.figure(figsize=(15, 10))
for i, class_name in enumerate(classes[:10]):
    class_path = os.path.join(dataset_path, class_name)
    image_files = os.listdir(class_path)[0]
    image_path = os.path.join(class_path, image_files)
    image = Image.open(image_path)

    plt.subplot(4, 5, i + 1)
    plt.imshow(image)
    plt.title(class_name)
    plt.axis('off')


plt.show()

In [ ]:
# %%
# Purpose: Preprocess the data using ImageDataGenerator.
# Result: Creates training and validation data generators for loading and augmenting images.
# Define image size and batch_size - Trained model - MobileNetV2 - 224 x 224
img_size = (224, 224)
batch_size = 64 # There can be 64 images in a batch

# Data Augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

# Create training and validation generators
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
# %%
# Purpose: Get class indices and names.
# Result: Prints the class indices and class names.
class_indices = train_generator.class_indices
print(class_indices)

class_names = list(class_indices.keys())
print("Class Names:", class_names)

In [ ]:
# %%
# Purpose: Build and compile the model.
# Result: Creates and compiles the model for training.
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
# %%
# Purpose: Train the model.
# Result: Trains the model and stores the training history.
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs = 20
)

In [ ]:
# %%
# Purpose: Plot training and validation accuracy/loss.
# Result: Generates plots showing the training and validation accuracy and loss over epochs.
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# %%
# Purpose: Evaluate the model on the validation set.
# Result: Prints the validation loss and accuracy.
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')

In [ ]:
# %%
# Purpose: Save the trained model.
# Result: Saves the model to a file named 'MCAR.keras'.
model.save('MCAR.keras')

In [ ]:
# %%
# Purpose: Load the saved model and define a prediction function.
# Result: Loads the model and defines a function to predict the animal in an image.
model = load_model('MCAR.keras')

def predict_animal(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)
    predicted_class = class_names[np.argmax(prediction)]

    plt.imshow(img)
    plt.title(f"Predicted: {predicted_class}")
    plt.axis('off')
    plt.show()

In [ ]:
# %%
# Purpose: List files in a specific directory.
# Result: Prints the list of files in the specified directory.
os.listdir('/kaggle/input/animal-image-dataset-90-different-animals/animals/animals/snake')

In [ ]:
# %%
# Purpose: Predict the animal in an image using the predict_animal function.
# Result: Predicts the animal in the specified image and displays the image with the prediction.
#predict_animal('/kaggle/input/animal-image-dataset-90-different-animals/animals/animals/parrot/0100ec7408.jpg')